In [160]:
#CLUSTER GORDIAS LIS 
import pyTigerGraph as tg
import pandas as pd
import gradio as gr
hostName = "https://4da1bd47ec034979919364bf0615f5f4.i.tgcloud.io"
graphName = "VWG"
secret = "uoabr03s0ls2prn9hpdupch6vquvn6gu"
userName = "user_1"
password = "A1z2e3r4*"
tigergraph_insights_map = "https://tools.tgcloud.io/insights/app/67eZLiuqbdk5nNg9QxWQfA/page/gvxC91DiSnRKvPvWzfR9iH/widgetShare/sQFKgLZh8iCgVFrarvUVTr?domain=4da1bd47ec034979919364bf0615f5f4.i&orgName=lis-gordias&clusterid=5fc3b84f-33fb-41b4-b856-2e983aa290f6&TigerGraphToken=3ff2a285-25b4-4a77-98c2-80bfbde3fd73"
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"secret token: {authToken}")
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password, apiToken=authToken)

secret token: h6orb4jpm767rofcvvk18ig6m953prpj


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\pyTigerGraph\pyTigerGraphBase.py:105: DeprecationWarning: The `apiToken` parameter is deprecated; use `getToken()` function instead.
  warnings.warn(


In [161]:
# # FREE CLUSTER EDGE
import pyTigerGraph as tg
import pandas as pd
import gradio as gr

hostName1 = "https://56dad41572d543beb9b2025cceeb23a3.i.tgcloud.io"
graphName1 = "VWG1"
secret1 = "fddg5vos93n1jb6sj6aio977tmjasuvv"
userName1 = "user_1"
password1 = "A1z2e3r4*"
tigergraph_insights_map1 = "https://tools.tgcloud.io/insights/app/8onBWB6MYoVRFmLp3Ath6F/page/1pUmxvpSfqoRc7vWAGwQdo/widgetShare/cZ6ovMqYG3ysiqpJd6XmXx?domain=56dad41572d543beb9b2025cceeb23a3.i&orgName=jrlisdata-org-2024325&clusterid=695408c0-9981-4988-af3b-114001e1c502&TigerGraphToken=1c649a54-731f-48f2-8d74-7ddc303580da"
graph1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1)

authToken1 = graph1.getToken(secret1)
authToken1 = authToken1[0]
print(f"secret token: {authToken1}")
conn1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1, username=userName1, password=password1, apiToken=authToken1)

secret token: dnl708mm0hclh9j45ag9vl3j8k30l14m


In [162]:
# deleted = conn1.upsertEdge(sourceVertexType="Nodes",targetVertexType="Nodes",
#                         sourceVertexId="Las Palmas de Gran Canaria",targetVertexId="Seville",edgeType="distribute_to")

# Tab 0 (cargar datos)

In [163]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
atributes_nodes = {}
atributes_edges = {}

# Helper functions
def path_taken(results):
    edges = results[2]['@@display_edge_set']
    node_initial = None
    edge_to_ids = {edge['to_id'] for edge in edges}
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break
    if node_initial is None:
        raise ValueError("initial node cannot be determined.")
    current_node = node_initial
    path = [current_node]
    while True:
        next_node = None
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        if next_node is None:
            break
        path.append(next_node)
        current_node = next_node
    nodes = {node_name: node_name for node_name in path}
    ordered_nodes = [nodes[node_name] for node_name in path]
    return ordered_nodes

def add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch , charge):
    existing_cities = [city for city, _ in nodes_batch_full]
    for node, _ in nodes_batch:
        if node in existing_cities:
            # search if town already in nodes_batch_full
            for i, (city, stock) in enumerate(nodes_batch_full):
                if city == node:
                    # update stock
                    if i == 0:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] - charge})
                    else:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] + charge})
            continue
        else:
            # add new entries to nodes_batch_full
            nodes_batch_full.extend([
                (nodes_batch[0][0], {"Stock": nodes_batch[0][1]["Stock"]}),
                (nodes_batch[1][0], {"Stock": nodes_batch[1][1]["Stock"]})
            ])
            return nodes_batch_full
    return nodes_batch_full

def add_or_update_final_edges_batch(edges_batch_full, edges_batch, charge):
    existing_edges = [(edge[0], edge[1]) for edge in edges_batch_full]
    
    for edge in edges_batch:
        # edge already exist ? 
        if (edge[0], edge[1]) in existing_edges or (edge[1], edge[0]) in existing_edges:
            # if yes, search if edge already in edges_batch_full
            for i, (origin, destination, movement) in enumerate(edges_batch_full):
                if (origin, destination) == (edge[0], edge[1]) or (origin, destination) == (edge[1], edge[0]):
                    # Update
                    edges_batch_full[i] = (origin, destination, {"Daily_movement": edges_batch_full[i][2]["Daily_movement"] + charge})
            continue
        else:
            # If no, add new entries to edges_batch_full
            edges_batch_full.append((edge[0], edge[1], {"Daily_movement": edge[2]["Daily_movement"]}))
    
    return edges_batch_full

# Main transfer function
def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full
    results_astar = conn.runInstalledQuery("tg_astar_test", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })
    
    order_taken = path_taken(results_astar)
    edge_set = results_astar[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    node_set = results_astar[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        if charge > node_i_stock:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente inventario ({charge} > {node_i_stock})\n"
            log_list += error_message

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
            log_list += error_message

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
            log_list += error_message

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        if daily_movement + charge > capacity_edge:
            error_message = f"Enviar {capacity_edge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
            log_list += error_message

        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost += element["attributes"]["Price"]

        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)
        operation_count += 1
        palets_number += charge

        return nodes_batch , edges_batch

    except Exception as e:
        error_message = f"Error en la transferancia: {e}\n"
        log_list += error_message
        return error_message

import random 
import string
def generate_random_title(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def generate_html():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="875" height="800" src="{tigergraph_insights_map}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="800" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html800():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="700" height="800" src="{tigergraph_insights_map}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1_800():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="700" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1000():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="1000" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """


def reset_html():
    new_html = generate_html()
    return gr.HTML(value=new_html, visible=True)

def reset_html1():
    new_html = generate_html1()
    return gr.HTML(value=new_html, visible=True)

def reset_html1_1000():
    new_html = generate_html1000()
    return gr.HTML(value=new_html, visible=True)


def reset_html_800():
    new_html = generate_html800()
    return gr.HTML(value=new_html, visible=True)

def reset_html1_800():
    new_html = generate_html1_800()
    return gr.HTML(value=new_html, visible=True)


# Thread safety
lock = threading.Lock()


def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full,alternatives_paths
    operation_count = 0
    palets_number = 0
    palets_cost = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full = []
    edges_batch_full = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        global log_list
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            nodes_batch, edges_batch = transfert_nodes_and_edges(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)

    if log_list == " ":
        log_list = "All operations were successful."

    return (log_list, reset_html(), gr.Button("Regargar mapa",elem_classes="otherbutton",visible=True),
        gr.Number(label="KPI : Numero de operaciones", value = operation_count,visible=True,elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de palets",value = palets_number, visible=True,elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de origen",value = len(unique_origin_nodes), visible=True,elem_classes="dropdown"), 
        gr.Number(label="KPI : Destinaciones diferentes",value = len(unique_destination_nodes), visible=True,elem_classes="dropdown"), 
        gr.Number(label="KPI : Coste total",value = palets_cost, visible=True,elem_classes="dropdown"), 
        gr.Number(label="KPI : Coste por palet",value = round(palets_cost/palets_number,1), visible=True,elem_classes="dropdown"))


def make_daily_movements1(input_file, column_origin, column_destination, column_transfert):
    global operation_count1, palets_number1, palets_cost1, unique_origin_nodes1, unique_destination_nodes1, log_list1, nodes_batch_full1
    operation_count1 = 0
    palets_number1 = 0
    palets_cost1 = 0
    unique_origin_nodes1 = set()
    unique_destination_nodes1 = set()
    log_list1 = " "
    nodes_batch_full1 = []
    edges_batch_full1 = []

    df = pd.read_csv(input_file)

    def perform_transfer1(row):
        global log_list1
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            nodes_batch, edges_batch = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full1, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full1, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} to {destination}, No camino existante.\n"
            log_list1 += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer1, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full1)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full1)
    return (log_list1,
        gr.Markdown("# Mapa red original",visible=True),reset_html_800(),gr.Button("Regargar mapa",visible=True,elem_classes='otherbutton'),
        gr.Markdown("# Mapa red modificada",visible=True),reset_html1_800(), gr.Button("Regargar mapa",visible=True,elem_classes='otherbutton'),
        # KPI RED ORIGINAL
        gr.Markdown("# Red original",visible=True,elem_classes='cabecero'),
        gr.Number(label="Cantidad de Aristas",value=numEdges,visible=True , elem_classes="dropdown"),                                 
        gr.Number(label="Cantidad de Nodos",value = numNodes,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad media por arista",value = avg_capacity,visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad total de las aristas ",value = total_capacity_edges,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad total de los Almacenes",value = total_capacity_warehouse,visible=True, elem_classes="dropdown"),
        gr.Number(label="Stock total de los almacenes",value = total_stock,visible=True, elem_classes="dropdown"),
        # KPI ENTREGA ORIGINAL
        gr.Markdown("# Entrega original",visible=True,elem_classes='cabecero'),
        gr.Number(label="Numero de operaciones", value = operation_count,visible=True, elem_classes="dropdown"),                    
        gr.Number(label="Numero de palets",value = palets_number, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Numero de origen",value = len(unique_origin_nodes), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Destinaciones diferentes",value = len(unique_destination_nodes), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste total",value = palets_cost, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste por palet",value = round(palets_cost/palets_number,1), visible=True, elem_classes="dropdown"),
        # KPI RED SIMULADA
        gr.Markdown("# Red modificada",visible=True,elem_classes='cabecero'),
        gr.Number(label="Cantidad de Aristas",value=numEdges1,visible=True, elem_classes="dropdown"),                                 
        gr.Number(label="Cantidad de Nodos",value = numNodes1,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad media por arista",value = avg_capacity1,visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad total de las aristas ",value = total_capacity_edges1,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad total de los Almacenes",value = total_capacity_warehouse1,visible=True, elem_classes="dropdown"),
        gr.Number(label="Stock total de los almacenes",value = total_stock1,visible=True, elem_classes="dropdown"),
        # KPI ENTREGA SIMULADA
        gr.Markdown("# Entrega modificada",visible=True,elem_classes='cabecero'),
        gr.Number(label="Numero de operaciones", value = operation_count1,visible=True, elem_classes="dropdown"),                    
        gr.Number(label="Numero de palets",value = palets_number1, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Numero de origen",value = len(unique_origin_nodes1), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Destinaciones diferentes",value = len(unique_destination_nodes1), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste total",value = palets_cost1, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste por palet",value = round(palets_cost1/palets_number1,1), visible=True, elem_classes="dropdown"))



def transfert_nodes_and_edges1(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count1, palets_number1, palets_cost1, unique_origin_nodes1, unique_destination_nodes1, log_list1, nodes_batch_full1
    results_astar = conn1.runInstalledQuery("tg_astar_test", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })
    
    order_taken = path_taken(results_astar)
    edge_set = results_astar[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    node_set = results_astar[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        if charge > node_i_stock:
            error_message = f"{node_initial} hasta {node_final}: No hay suficientes inventario ({charge} > {node_i_stock})\n"
            log_list += error_message

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
            log_list += error_message

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
            log_list += error_message

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        if daily_movement + charge > capacity_edge:
            error_message = f"Enviar {capacity_edge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
            log_list1 += error_message

        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost1 += element["attributes"]["Price"]

        unique_origin_nodes1.add(node_initial)
        unique_destination_nodes1.add(node_final)
        operation_count1 += 1
        palets_number1 += charge

        return nodes_batch , edges_batch

    except Exception as e:
        error_message = f"Error en la transferancia: {e} \n"
        log_list1 += error_message
        return error_message

# log_messages = make_daily_movements1(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\df_1_move.csv", "CODE_Origin", "CODE_Destination", "Palets")

# Functions tab 1 ( KPI red 2 )

# Functions tab 2 (modif red)

In [164]:
import json
global logs_modif_network
logs_modif_network = " "


# ADD NODES 

def add_node(id, attributes_dict=None, vertex_type="Nodes"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertVertex(vertex_type, id)
        message =  f"Nodo {id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertVertex(vertex_type, id, attributes=attributes)
        message =  f"Nodo {id} añadido with attributos : {attributes} \n"
        logs_modif_network +=message
    return logs_modif_network

def get_values_nodes(*args):
    values = {key: val for key, val in zip(atributes_nodes.keys(), args)}
    return values

def get_values_edges(*args):
    values = {key: val for key, val in zip(atributes_edges.keys(), args)}
    return values["Capacity"]


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    return add_node(id, attributes_json), reset_html1()



# ADD EDGES 
def add_new_edge(source_vertex_id, target_vertex_id, attributes_dict=None,source_vertex_type="Nodes",target_vertex_type = "Nodes", edge_type="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id}  añadida"
        logs_modif_network += message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id} añadida con los atributos siguientes: {attributes} \n"
        logs_modif_network += message
    return logs_modif_network

def add_edge_with_values(source_vertex_id, target_vertex_id, *args):
    attributes = get_values_edges(*args)
    attributes_json = json.dumps(attributes)
    return add_new_edge(source_vertex_id,target_vertex_id, attributes_json), reset_html1()

# GET ATTRIBUTES
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn1.getVertexAttrs('Nodes')
conex_edges_list = conn1.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])

for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])



# GET NODES NAMES
list_name_nodes = []
name_nodes_conn = conn1.getVertices("Nodes")
for i in range(0, len(name_nodes_conn)):
    list_name_nodes.append(name_nodes_conn[i]["v_id"])

def add_edge(source_vertex_id, target_vertex_id, attributes_dict=None, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id)
        message =  f"Arista desde {source_vertex_id} hasta {target_vertex_id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id,
                        attributes=attributes)
        message =  f"Arista {source_vertex_id} hasta {target_vertex_id} añadido con los atributos siguientes : {attributes} \n"
        logs_modif_network +=message
        return logs_modif_network


#Erase Data

def erase_edge(source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    result = conn1.delEdges(sourceVertexType=vertex_type,
                           targetVertexType=vertex_type,
                           edgeType=edgeType,
                           sourceVertexId=source_vertex_id,
                           targetVertexId=target_vertex_id)
    
    if result == {edgeType: 0}:
        message = f"Arista de {source_vertex_id} hacia {target_vertex_id} no existe o no ha podido borrarse\n"
    else:
        message = f"Arista de  {source_vertex_id} hacia {target_vertex_id} borrada \n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()

    


def erase_node(id, vertex_type="Nodes"):
    global logs_modif_network
    result = conn1.delVerticesById(vertexIds=id, vertexType=vertex_type)
    
    if result == 0:
        message = f"Nodo {id} no existe o no ha podido borrarse\n"
    else:
        message = f"Nodo {id} borrado\n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()




# add_edge_with_values("Las Palmas de Gran Canaria","Seville",{"Capacity": 0, "Daily_movement": 0, "Price": 0})



In [165]:
# source_vertex_type = "Nodes"
# source_vertex_id = "Las Palmas de Gran Canaria"
# target_vertex_type = "Nodes"
# target_vertex_id = "Seville"
# edge_type = "distribute_to"
# attributes = {"Capacity": 1000, "Daily_movement": 0, "Price": 0.5}
# result = conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)

In [166]:
# result

In [167]:
# conn.delEdges(sourceVertexType="Nodes",
#                            targetVertexType="Nodes",
#                            edgeType="distribute_to",
#                            sourceVertexId="Badalona",
#                            targetVertexId="Barcelona")

In [168]:
# conn.delVerticesById(vertexIds="Oviedo",vertexType="Nodes")

# Function Tab 3 (algorithms)

In [169]:
# def launch_instaled_algo(algorithm,params_str):
#     params = json.loads(params_str)
#     results = conn.runInstalledQuery(algorithm, params= params)
#     return results

In [170]:
# VertexTypes = conn.getVertexTypes()
# EdgeTypes = conn.getEdgeTypes()
# EdgeAtributes = conn.getEdgeAttrs("distribute_to")[0][0]
# data_algo = conn.getInstalledQueries(fmt="py")
# installed_algorithms = [key.split("/")[-1] for key in data_algo.keys() if key.startswith("GET /query/VWG/")]

# Tab 4 

In [171]:
# import matplotlib.pyplot as plt
# import networkx as nx
# from io import BytesIO
# from PIL import Image

# def networkx_maxflow(data):
#     data = eval(data)
#     max_flow = data[0]['@@sum_max_flow']
#     edge_set = data[1]['@@edges_set']

#     G = nx.DiGraph()

#     # Add edges with capacities
#     for edge in edge_set:
#         from_node = edge['from_id']
#         to_node = edge['to_id']
#         capacity = edge['attributes']['Capacity']
#         G.add_edge(from_node, to_node, capacity=capacity)

#     pos = nx.spring_layout(G)
#     nx.draw(G, pos, with_labels=True, node_size=700, node_color='lightblue', font_size=10, font_weight='bold')  # Draw nodes
#     nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}, font_color='red')  # Edge labels
#     plt.title(f"Max Flow: {max_flow}")
#     plt.show()

#     buf = BytesIO()
#     plt.savefig(buf, format='png')
#     buf.seek(0)
#     img = Image.open(buf)
#     return img

In [172]:
# import matplotlib.pyplot as plt
# import networkx as nx
# def networkx_astar(data):
#     data = eval(data)
#     node_data = {item['v_id']: item['attributes'] for item in data[2]['tmp']}
#     edge_data = [{'from': edge['from_id'], 'to': edge['to_id'], 'capacity': edge['attributes']['Capacity']} for edge in data[2]['@@display_edge_set']]

#     G = nx.Graph()

#     # Add nodes
#     for node_id, attributes in node_data.items():
#         G.add_node(node_id, **attributes)

#     # Add edges
#     for edge in edge_data:
#         G.add_edge(edge['from'], edge['to'], capacity=edge['capacity'])

#     pos = nx.spring_layout(G)  # Nodes
#     nx.draw(G, pos, with_labels=True, node_size=700, node_color='lightblue', font_size=10, font_weight='bold')  # Draw Edges
#     nx.draw_networkx_edge_labels(G, pos, edge_labels={(edge['from'], edge['to']): f"{edge['capacity']}" for edge in edge_data}, font_color='red')  # Edge label
#     plt.title("Graphe avec NetworkX")
#     plt.show()
#     from io import BytesIO
#     from PIL import Image
#     buf = BytesIO()
#     plt.savefig(buf, format='png')
#     buf.seek(0)
#     img = Image.open(buf)
#     return img

In [173]:
def read_csv_path(input_file):
    df = pd.read_csv(input_file.name)
    target_column = sorted(list(df.columns))
    return  gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column)

Value first tab

In [174]:
name_edge = "distribute_to"
name_node = "Nodes"


#######RED ORIGINAL######

numEdges = conn.getEdgeCount(name_edge)
numNodes = conn.getVertexCount(name_node)
edge = conn.getEdgeStats(name_edge)
avg_capacity = round(edge[name_edge]['Capacity']['AVG'],1)

edges = conn.getEdgesByType(name_edge)
total_capacity_edges = 0
for warehouse in edges :
    capacity_edges = warehouse["attributes"]["Capacity"]     
    total_capacity_edges += round(capacity_edges,1)

almacenes = conn.getVertices(name_node)
total_capacity_warehouse = 0
for warehouse in almacenes :
    capacity_edges = warehouse["attributes"]["Capacity"]      
    total_capacity_warehouse += round(capacity_edges,1)
    total_capacity_warehouse = round(total_capacity_warehouse,1)

almacen = conn.getVertices(name_node)
total_stock = 0
for warehouse in almacen :
    stock = warehouse["attributes"]["Stock"]     
    total_stock += stock
    total_stock = round(total_stock,0)

capacity_per_edge = round(total_capacity_warehouse/numEdges,1)


#######RED SIMULADA######

numEdges1 = conn1.getEdgeCount(name_edge)

numNodes1 = conn1.getVertexCount(name_node)

edges1 = conn1.getEdgeStats(name_edge)
avg_capacity1 = edges1[name_edge]['Capacity']['AVG']
avg_capacity1 =round(avg_capacity1,1)

edge = conn1.getEdgesByType(name_edge)
total_capacity_edges1 = 0
for warehouse in edge :
    capacity_edge1 = warehouse["attributes"]["Capacity"]     
    total_capacity_edges1 += capacity_edge1

almacen1 = conn1.getVertices(name_node)
total_capacity_warehouse1 = 0
for warehouse in almacen1 :
    capacity_nodes1 = warehouse["attributes"]["Capacity"]     
    total_capacity_warehouse1 += capacity_nodes1
    total_capacity_warehouse1 = round(total_capacity_warehouse1,1)


almacen1 = conn1.getVertices(name_node)
total_stock1 = 0
for warehouse in almacen1 :
    stock = warehouse["attributes"]["Stock"]     
    total_stock1 += stock
    total_stock1 = round(total_stock1,0)


capacity_per_edge1 = round(total_capacity_warehouse1/numEdges1,0)

In [186]:
import gradio as gr

class GradioMetro:
    
    def front_func(self):
        with gr.Blocks(css="stylessupersimple.css", title ="Red de distribución",theme="monochrome") as demo:
            gr.Markdown("## ""![](file/Gradio/logo_lis.svg) Red de distribución""", elem_classes="cabecero")
            with gr.Tabs() as tabs:

                with gr.TabItem("KPIs red",id=1):
                    with gr.Row():
                        with gr.Column(scale=5):
                            map_tiger = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="1000" height="800"
                                    src=""" + tigergraph_insights_map1 + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")
                            btn_reload_map =  gr.Button("Regargar mapa",elem_classes="otherbutton")
                            btn_reload_map.click(fn=reset_html1_1000,outputs=[map_tiger])  
                        with gr.Column(scale=2):
                            gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,elem_classes='textbox')
                            gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,elem_classes='textbox')
                            gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,elem_classes='textbox')
                            gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,elem_classes='textbox')
                
                with gr.TabItem("Cargar movimientos",id=2):
                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')

                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True ,elem_classes='textbox')
                            load_data_send = gr.Button("Lanzar los envios",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])
                    with gr.Row():
                        logs = gr.Textbox(label="Logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=10):
                            map_entrega = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""", visible = False
                                    )
                            btn_reload_map2 = gr.Button("Regargar mapa",elem_classes="otherbutton",visible=False)
                            btn_reload_map2.click(fn=reset_html,outputs=[map_entrega])  
                        with gr.Column(scale=6):
                  
                            operation_number = gr.Number(label="KPI : Numero de operaciones", visible=False)
                            palets_number = gr.Number(label="KPI : Numero de palets", visible=False)
                            origin_number = gr.Number(label="KPI : Numero de origen", visible=False)
                            distinct_destinations = gr.Number(label="KPI : Destinaciones diferentes", visible=False)
                            total_cost = gr.Number(label="KPI : Coste total", visible=False)
                            total_cost_per_palet = gr.Number(label="KPI : Coste por palet", visible=False)
                            # button_KPI = gr.Button("Calcular KPIs",visible=False)
                            # button_KPI.click(get_KPIs_movements , outputs= [operation_number,palets_number,origin_number,distinct_destinations,total_cost,total_cost_per_palet]) 

                        load_data_send.click(make_daily_movements,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                             outputs=[logs,map_entrega,btn_reload_map2,operation_number,palets_number,origin_number,distinct_destinations,total_cost,total_cost_per_palet])

                with gr.TabItem("Escenario What if", id=3):
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("# Añadir/modificar nodo",elem_classes="cabecero")
                            new_node_id = gr.Textbox(label="Nombre nuevo nodo",elem_classes="dropdown")
                            for e in attributes_nodes_list:
                                with gr.Row():
                                    atributes_nodes[e] = gr.Number(label=e,elem_classes="dropdown")
                        
                            with gr.Row():
                                btn_modif_nodes = gr.Button("Añadir/modificar nodos",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Añadir/modificar arista",elem_classes="cabecero")

                            source_vertex_id_edge = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",elem_id = "source_vertex")
                            target_vertex_id_edge = gr.Textbox(label="Nodo de destino",elem_classes="dropdown")

                            for e in attributes_edges_list:
                                with gr.Row():
                                    atributes_edges[e] = gr.Number(label=e,elem_classes="dropdown")
                            btn_modif_edge = gr.Button("Añadir/modificar arista",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Borrar nodo/arista",elem_classes="cabecero")
                            source_vertex_id = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",container = True)
                            target_vertex_id = gr.Textbox(label="Nodo de destino (si es una arista)",elem_classes="dropdown",container = True)
                            btn_erase_node = gr.Button("Borrar nodo",elem_classes="otherbutton")
                            btn_erase_edge = gr.Button("Borrar arista",elem_classes="otherbutton")  

                    with gr.Row():
                        logs = gr.Textbox(label = "logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')
                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            load_data_send1 = gr.Button("Lanzar los envios en la red modificada",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])

                    with gr.Row():
                        logs1 = gr.Textbox(label = "logs",elem_classes="dropdown")

                            
                    with gr.Row():
                        with gr.Column():
                            markdown1 = gr.Markdown("# KPIs red original",visible=False)
                            num_edges_number1 = gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,visible=False)
                            num_nodes_number1 = gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,visible=False)
                            capacity_average_number1 = gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,visible=False)
                            total_capacity_edge_number1 = gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,visible=False)
                            total_capacity_warehouse_number1 = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,visible=False)
                            total_stock_number1 = gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,visible=False)
                            ####KPI ENTREGA RED ORGINAL####
                        with gr.Column():
                            markdown2 = gr.Markdown("# KPIs Entrega original",visible=False)
                            operation_number1 = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number1 = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number1 = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations1 = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost1 = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet1 = gr.Number(label= "KPI : Coste por palet",visible=False)
                        with gr.Column():
                            ###KPI RED MODIFICADA####
                            markdown1_simul =gr.Markdown("# KPIs red modificada",visible=False)
                            num_edges_number_simul = gr.Number(label="KPI : Cantidad de Aristas",value=numEdges1,visible=False)
                            num_nodes_number_simul = gr.Number(label="KPI : Cantidad de Nodos",value = numNodes1,visible=False)
                            capacity_average_number_simul = gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity1,visible=False)
                            total_capacity_edge_number_simul = gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges1,visible=False)
                            total_capacity_warehouse_number_simul = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse1,visible=False)
                            total_stock_number_simul = gr.Number(label="KPI : Stock total de los almacenes",value = total_stock1,visible=False)
                        with gr.Column():
                            markdown2_simul =gr.Markdown("# KPIs entrega red modificada",visible=False)
                            operation_number_simul = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number_simul = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number_simul = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations_simul = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost_simul = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet_simul = gr.Number(label= "KPI : Coste por palet",visible=False)



                    with gr.Row():
                        with gr.Column():
                            markdown_mapa_original =gr.Markdown("# Mapa red original",visible=False)
                            map_entrega_original = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_original = gr.Button("Recargar mapa",visible=False)

                        with gr.Column():
                            markdown_mapa_modificada =gr.Markdown("# Mapa modificada",visible=False)
                            map_entrega_simulacion = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map1 + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_simul = gr.Button("Recargar mapa",visible=False)


                        load_data_send1.click(make_daily_movements1,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                            outputs=[logs1,markdown_mapa_original,map_entrega_original,btn_reload_map_original,markdown_mapa_modificada,map_entrega_simulacion,btn_reload_map_simul,
                                                    markdown1,num_edges_number1,num_nodes_number1,capacity_average_number1,total_capacity_edge_number1,total_capacity_warehouse_number1,total_stock_number1,   #Red original
                                                    markdown2,operation_number1,palets_number1,origin_number1,distinct_destinations1,total_cost1,total_cost_per_palet1,   #Entrega original
                                                    markdown1_simul,num_edges_number_simul,num_nodes_number_simul,capacity_average_number_simul,total_capacity_edge_number_simul,total_capacity_warehouse_number_simul,total_stock_number_simul, #Red simulada
                                                    markdown2_simul,operation_number_simul,palets_number_simul,origin_number_simul,distinct_destinations_simul,total_cost_simul,total_cost_per_palet_simul]
                                                    , scroll_to_output=True) #Entrega simulada 
                            
                               
                    inputs_edge = [source_vertex_id_edge,target_vertex_id_edge] + [atributes_edges[e] for e in attributes_edges_list]
                    
                    inputs_nodes = [new_node_id] + [atributes_nodes[e] for e in attributes_nodes_list]
                    
                    def add_edge_with_values_wrapper(source, target, *args):
                        attributes = {e: arg for e, arg in zip(attributes_edges_list, args)}
                        return add_edge_with_values(source, target, attributes)

                    btn_modif_nodes.click(fn=add_node_with_values, inputs=inputs_nodes, outputs=[logs,map_entrega_simulacion])
                    btn_modif_edge.click(fn=add_edge_with_values_wrapper, inputs=inputs_edge, outputs=[logs,map_entrega_simulacion])
                    btn_erase_node.click(fn=erase_node, inputs=[source_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_erase_edge.click(fn=erase_edge, inputs=[source_vertex_id,target_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_reload_map_original.click(fn=reset_html_800,outputs=[map_entrega_original])
                    btn_reload_map_simul.click(fn=reset_html1_800,outputs=[map_entrega_simulacion])


                # with gr.TabItem("KPIs red modificada",id=6):
                #     with gr.Row():
                #         with gr.Column(scale=10):
                #             gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                #                     initial-scale=1.0"></head><body><iframe width="801" height="801"
                #                     src=""" + tigergraph_insights_map + """ 
                #                     title="tigergarph insights" frameborder="0"></iframe></body></html>""")   
                #         with gr.Column():
                #             Numedges_modif = gr.Number(label="KPI : Cantidad de Aristas")
                #             Numnodes_modif = gr.Number(label="KPI : Cantidad de Nodos")
                #             average_cost_per_edge_modif = gr.Number(label= "KPI : Coste medio / Arista")
                #             capacidad_total_aristas_modif = gr.Number(label="KPI : Capacidad total de las aristas ")
                #             warehouse_total_capacity_modif = gr.Number(label= "KPI : Capacidad total de los Almacenes")
                #             warehouse_total_stock_modif = gr.Number(label="KPI : Stock total de los almacenes")
                #             button_modif = gr.Button("Calcular KPIs")
                #             button_modif.click(get_KPIs_network , outputs= [Numedges_modif,Numnodes_modif,average_cost_per_edge_modif,capacidad_total_aristas_modif,warehouse_total_capacity_modif,warehouse_total_stock_modif]) 

                            

                demo.launch(max_threads=1000)
                
if __name__ == "__main__":

    lis = GradioMetro()
    
    lis.front_func()

Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\routes.py:1150: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\starlette\templating.py:178: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
